<p><b>Conexión a Twitter</b></p>

In [177]:
from twython import Twython
APP_KEY = ''
APP_SECRET = ''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

<p><b>Para eliminar los nodos existentes</b></p>

In [207]:
from py2neo import Graph
g = Graph(auth=('user', 'pass'))
g.delete_all()

<p><b>Crea los nodos a partir de los datos de twiter</b></p>
<img src="1.png" width="600" height="600">

In [208]:
#nodo persona esta relacionado con los nodos tweet(Person --"TWEETS"-> Tweet)

import pprint
from py2neo import Graph, Node, Relationship, NodeMatcher
results = twitter.search(q="#valdiviacl", count=100, result_type="recent")

for tweet in results['statuses']:
    
    fecha = tweet["created_at"]
    text = tweet['text']
    user_name = tweet['user']['name']
    location = tweet['user']['location']
    retweet = tweet['retweet_count']
    favorite = tweet['favorite_count']
    _id = tweet['user']['id']
    _idt = tweet['id']
    
    g = Graph(auth=('sgaete', 'sgaete1'))
    matcher = NodeMatcher(g)
    person = matcher.match("Person", _id=_id).first()
    tx = g.begin()
    if(person is None):
        person = Node("Person", _id=_id, name=user_name, location=location)
        tx.create(person)
    tweet_ = Node("Tweet", text=text, retweet_n=retweet, fav_n=favorite, fecha=fecha, _id=_idt)
    tx.create(tweet_)
    rel = Relationship(person, "TWEETS", tweet_)
    tx.create(rel)
    tx.commit()



<p><b>Ranking de tweets por retweets</b></p>

In [211]:
best = g.run("MATCH (n:Tweet) RETURN n ORDER BY n.nbRet DESC").data()
print("La persona con más retweets es:\n")
pp(g.run("MATCH (Tweet { _id: %s })--(Person) RETURN Person.name" % best[0]['n']['_id']).data())
print("\n-----------------------------------------------------------------------")
pp(best)



La persona con más retweets es:

[{'Person.name': 'Rod Naranjo'}]

-----------------------------------------------------------------------
[{'n': {'_id': 1029795780566155265,
        'fav_n': 0,
        'fecha': 'Wed Aug 15 18:23:41 +0000 2018',
        'retweet_n': 39,
        'text': 'RT @ObrequeMauricio: En #ValdiviaCl somos tan rascas que '
                'viene Piñera a inaugurar el Cau Cau, un fracaso de la '
                'ingeniería, el baluarte del…'}},
 {'n': {'_id': 1029798391126085632,
        'fav_n': 0,
        'fecha': 'Wed Aug 15 18:34:03 +0000 2018',
        'retweet_n': 39,
        'text': 'RT @ObrequeMauricio: En #ValdiviaCl somos tan rascas que '
                'viene Piñera a inaugurar el Cau Cau, un fracaso de la '
                'ingeniería, el baluarte del…'}},
 {'n': {'_id': 1029798363989004289,
        'fav_n': 0,
        'fecha': 'Wed Aug 15 18:33:57 +0000 2018',
        'retweet_n': 39,
        'text': 'RT @ObrequeMauricio: En #ValdiviaCl somos tan rasc

<p><b>Ranking de tweets por likes</b></p>

In [212]:
best = g.run("MATCH (n:Tweet) RETURN n ORDER BY n.nbLikes DESC").data()
print("La persona con más likes es:\n")
pp(g.run("MATCH (Tweet { _id: %s })--(Person) RETURN Person.name" % best[0]['n']['_id']).data())
print("\n-----------------------------------------------------------------------")
pp(best)

La persona con más likes es:

[{'Person.name': 'Rod Naranjo'}]

-----------------------------------------------------------------------
[{'n': {'_id': 1029795780566155265,
        'fav_n': 0,
        'fecha': 'Wed Aug 15 18:23:41 +0000 2018',
        'retweet_n': 39,
        'text': 'RT @ObrequeMauricio: En #ValdiviaCl somos tan rascas que '
                'viene Piñera a inaugurar el Cau Cau, un fracaso de la '
                'ingeniería, el baluarte del…'}},
 {'n': {'_id': 1029798391126085632,
        'fav_n': 0,
        'fecha': 'Wed Aug 15 18:34:03 +0000 2018',
        'retweet_n': 39,
        'text': 'RT @ObrequeMauricio: En #ValdiviaCl somos tan rascas que '
                'viene Piñera a inaugurar el Cau Cau, un fracaso de la '
                'ingeniería, el baluarte del…'}},
 {'n': {'_id': 1029798363989004289,
        'fav_n': 0,
        'fecha': 'Wed Aug 15 18:33:57 +0000 2018',
        'retweet_n': 39,
        'text': 'RT @ObrequeMauricio: En #ValdiviaCl somos tan rascas 